### Import Data

In [ ]:
import json
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

corpus = load_data("local_data.json")["File"]
# corpus[0]

### Clustering

In [ ]:
import os
import glob
import json
# import zstandard as zstd
import pandas as pd
import numpy as np

In [ ]:
# compressed_path = r"C:\Users\MoRevolution\Desktop\College\Data Dump\Reddit Sumbission Dump\reddit\submissions"
# temp_path = r"C:\Users\MoRevolution\Desktop\College\Data Dump\temp\RS_2005-06.zst"
decompressed_path = r"C:\Users\MoRevolution\Desktop\College\Data Dump\Reddit Sumbission Dump\reddit\submissions\Decompressed\RS_2018-06.json"
output_path = r"C:\Users\MoRevolution\Desktop\College\Data Dump\Reddit Sumbission Dump\reddit\submissions\Decompressed\chunks"
# listing = glob.glob(temp_path + '\\*.zst')

In [ ]:
chunk_size=50000
batch_no=1 

for chunk in pd.read_json(decompressed_path,lines=True,chunksize=chunk_size):
    chunk.to_csv(os.path.join(output_path, str(batch_no)+'.csv'),index=False)
    batch_no+=1

In [ ]:
path = r"C:\Users\MoRevolution\Desktop\College\Data Dump\Reddit Sumbission Dump\reddit\submissions\Decompressed\chunks\1.csv"
df = pd.read_csv(path)
arr = np.array(["college", "AskReddit", "science", "psychology", "socialwork", "personalfinance", "financialaid"])
count = (df['subreddit'].isin(arr)).sum()


In [ ]:
counts = df['subreddit'].value_counts()[arr]
print(counts)

In [ ]:
df.columns

## Filtering

### Heuristic Filtering

In [ ]:

g_story = []
# Define the keywords and phrases 
relevant_keywords = ['low socio-economic background', 'higher education', 'difficulties', 'overcome']

# Define the keywords and phrases that are likely to appear in irrelevant stories
irrelevant_keywords = ['high-achieving', 'wealthy', 'privileged']


for story in corpus: 
    # Check if the story contains any of the relevant keywords
    relevant_count = sum(1 for keyword in relevant_keywords if keyword in story.lower())

    # Check if the story contains any of the irrelevant keywords
    irrelevant_count = sum(1 for keyword in irrelevant_keywords if keyword in story.lower())

    # If the story is relevant, print it
    if relevant_count > 0 and irrelevant_count == 0:
        g_story.append(story)


### Text Classification

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score



In [ ]:
labeled_data = corpus

# Split the data into training and testing sets
train_data = [data['text'] for data in labeled_data if data['label'] == 'relevant']
train_labels = [data['label'] for data in labeled_data if data['label'] == 'relevant']
test_data = [data['text'] for data in labeled_data if data['label'] != 'relevant']
test_labels = [data['label'] for data in labeled_data if data['label'] != 'relevant']

# Convert the text data into a bag-of-words representation
vectorizer = CountVectorizer(stop_words='english')
train_data_counts = vectorizer.fit_transform(train_data)
test_data_counts = vectorizer.transform(test_data)

In [ ]:
# Train a Naive Bayes classifier on the labeled data
clf = MultinomialNB()
clf.fit(train_data_counts, train_labels)

# Evaluate the classifier on the test data
predictions = clf.predict(test_data_counts)
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy}")

# Filter the relevant stories from the corpus
with open('corpus.json', 'r') as f:
    corpus = json.load(f)

relevant_stories = []
for story in corpus:
    story_counts = vectorizer.transform([story['text']])
    if clf.predict(story_counts) == 'relevant':
        relevant_stories.append(story)

print(f"Found {len(relevant_stories)} relevant stories.")
